# Predecir nivel de adaptabilidad

In [2]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from keras import layers
from sklearn.preprocessing import OneHotEncoder

print(tf.__version__)

2023-11-20 10:27:17.266868: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-20 10:27:17.560261: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 10:27:17.560303: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 10:27:17.562254: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 10:27:17.715043: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-20 10:27:17.717341: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

2.14.0


## Exploracion de los datos
The data is obtained from the csv file "training-ds"

In [7]:
raw_dataset = pd.read_csv("./training-ds.csv")
dataset = raw_dataset.copy()
raw_dataset.head()


,Tipo de Red,Estudiante de Tecnología,Nivel de Educación,Vive en Ciudad,Tipo de Instituto,Edad,Dispositivo,Tipo de Internet,Situación Financiera,Género,Duración de la Clase,Nivel de Adaptación
0,3G,Si,Universidad,Si,Privado,21-25,Computadora,Wifi,Media,Masculino,03-Jun,Bajo
1,3G,No,Escuela,Si,Privado,Nov-15,Smartphone,Compra Megas,Media,Femenino,01-Mar,Moderado
2,3G,Si,Universidad,Si,Privado,21-25,Smartphone,Compra Megas,Mala,Masculino,01-Mar,Bajo
3,3G,Si,Escuela,Si,Privado,Nov-15,Smartphone,Compra Megas,Media,Masculino,01-Mar,Moderado
4,4G,Si,Universidad,Si,Privado,21-25,Computadora,Wifi,Buena,Masculino,01-Mar,Alto


### Cleaning Data

If the dataset contains any unknown value the row containing it will be deleted

In [8]:
dataset.isna().sum()

Tipo de Red                 0
Estudiante de Tecnología    0
Nivel de Educación          0
Vive en Ciudad              0
Tipo de Instituto           0
Edad                        0
Dispositivo                 0
Tipo de Internet            0
Situación Financiera        0
Género                      0
Duración de la Clase        0
Nivel de Adaptación         0
dtype: int64

As the dataset does not contain any unknown values, no row will be deleted.

### Correcting the data
The "Edad" column should contain values that indicate an age interval following this format: "21-24". 
But there are values that changed the value of a limit for a month like this: "Nov-15"

The data is corrected using the "correct_age" function.

In [16]:
def correct_number(edad): 
    months = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06', 
             'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

    for m in months: 
        if m in edad: 
            return edad.replace(m, months[m]) 
        
    return edad

This function is applied to the "Edad" column.

In [17]:
dataset['Edad'] = dataset["Edad"].apply(correct_number)
dataset.head()

,Tipo de Red,Estudiante de Tecnología,Nivel de Educación,Vive en Ciudad,Tipo de Instituto,Edad,Dispositivo,Tipo de Internet,Situación Financiera,Género,Duración de la Clase,Nivel de Adaptación
0,3G,Si,Universidad,Si,Privado,21-25,Computadora,Wifi,Media,Masculino,03-Jun,Bajo
1,3G,No,Escuela,Si,Privado,11-15,Smartphone,Compra Megas,Media,Femenino,01-Mar,Moderado
2,3G,Si,Universidad,Si,Privado,21-25,Smartphone,Compra Megas,Mala,Masculino,01-Mar,Bajo
3,3G,Si,Escuela,Si,Privado,11-15,Smartphone,Compra Megas,Media,Masculino,01-Mar,Moderado
4,4G,Si,Universidad,Si,Privado,21-25,Computadora,Wifi,Buena,Masculino,01-Mar,Alto


Same as the "Edad" column the "Duracion de la clase" column has the number values changed. 
The same function is applied to this column

In [18]:
dataset['Duración de la Clase'] = dataset["Duración de la Clase"].apply(correct_number)
dataset.head()

,Tipo de Red,Estudiante de Tecnología,Nivel de Educación,Vive en Ciudad,Tipo de Instituto,Edad,Dispositivo,Tipo de Internet,Situación Financiera,Género,Duración de la Clase,Nivel de Adaptación
0,3G,Si,Universidad,Si,Privado,21-25,Computadora,Wifi,Media,Masculino,03-06,Bajo
1,3G,No,Escuela,Si,Privado,11-15,Smartphone,Compra Megas,Media,Femenino,01-03,Moderado
2,3G,Si,Universidad,Si,Privado,21-25,Smartphone,Compra Megas,Mala,Masculino,01-03,Bajo
3,3G,Si,Escuela,Si,Privado,11-15,Smartphone,Compra Megas,Media,Masculino,01-03,Moderado
4,4G,Si,Universidad,Si,Privado,21-25,Computadora,Wifi,Buena,Masculino,01-03,Alto


### One hot encoding

In [ ]:

one_hot_encoder = OneHotEncoder(sparse=False)

one_hot_encoded = one_hot_encoder.fit_transform(dataset[['column_to_encode']])

one_hot_df = pd.DataFrame(one_hot_encoded, columns=one_hot_encoder.get_feature_names(['column_to_encode']))


# Models

# Training